In [2]:
# This file is aimed to add Xe to the model and see if the model can be improved
from src.DataPrepocessing import *
from src.NeuralNets import *
from src.TrainMethod import *

import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Data Propocessing

This part will generate a file with following parts
* Keff
* Power
* Threshold
* Histogramed Cs
* Histogramed Xe

In [3]:
# First, get the keff power and threshold from tracked_results.csv

core_file = pd.read_csv("./Raw Data/Add Xe/tracked_results.csv")
core_file.shape

(1309, 13)

In [4]:
extract_columns = ["keff", "power_normalization_value", "threshold"]
core_data = core_file[extract_columns].to_numpy()
core_data.shape

(1309, 3)

In [5]:
# get the histogramed version of Cs and Xe

def generate_histogram(data: np.ndarray, binInt = 1.0e-06, binMax = 1.5e-04):
    '''
    Take in a data with columns are features  Transform the features with histogram method
    '''

    # Extract data
    Xvals = data

    nRows, nCols = Xvals.shape

    # define bins
    hEdges = np.arange(0, binMax+binInt, binInt)
    nBins = len(hEdges)-1


    # fill histogram matrix
    nCountsX = np.zeros((nRows, nBins), dtype=np.uint16)
    for n in range(nRows):
        nCountsX[n,:], _ = np.histogram(Xvals[n,:], hEdges)
    return nCountsX

In [6]:
Cs137_file = pd.read_csv("./Raw Data/Add Xe/Cs137.csv").to_numpy()
Xe135_file = pd.read_csv("./Raw Data/Add Xe/Xe135.csv").to_numpy()


histogramed_Cs137 = generate_histogram(Cs137_file)
histogramed_Xe135 = generate_histogram(Xe135_file)


print(histogramed_Cs137.shape)
print(histogramed_Xe135.shape)

(1309, 150)
(1309, 150)


In [7]:
# do the time align
# although it does not have a clear effect on the result

core_data = core_data[1:]

histogramed_Cs137 = histogramed_Cs137[:-1]
histogramed_Xe135 = histogramed_Xe135[:-1]

print(core_data.shape)
print(histogramed_Cs137.shape)
print(histogramed_Xe135.shape)

(1308, 3)
(1308, 150)
(1308, 150)


In [9]:
total_data = np.concatenate((core_data, histogramed_Cs137, histogramed_Xe135), axis=1)
total_data.shape

(1308, 303)

In [10]:
# check if the first column is the keff

total_data[:,0]

array([0.97026873, 0.9686063 , 0.96600921, ..., 0.95942264, 0.95950929,
       0.95917451])

In [12]:
# Drop the data 

# Save the array as a CSV file
np.savetxt('./Processed Data/Cs_Xe.csv', total_data, delimiter=',')